# Update Stock Historical Price/Volume Database

## 資料檔案設定

In [1]:
data_filepath = 'data/data.feather'  # .feather 儲存格式

## 程式執行

### 套件設定

In [2]:
import os
import pandas as pd
import yfinance as yf
from getStockList import get_stock_symbols
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterSession, LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
import datetime
import pytz

In [3]:
# Rate limiter and cached
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

# start a cached session
session = CachedLimiterSession(
    limiter=Limiter(RequestRate(10, Duration.SECOND*2)),
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("requests.cache"),
)

### 載入資料

In [4]:
# Get stock list
stock_symbol_df = get_stock_symbols()

stock_symbol_df  # Inspect stock symbol df

,symbol,name,market,industry,yf_symbol
0,1101,台泥,上市,水泥工業,1101.TW
1,1102,亞泥,上市,水泥工業,1102.TW
2,1103,嘉泥,上市,水泥工業,1103.TW
3,1104,環泥,上市,水泥工業,1104.TW
4,1108,幸福,上市,水泥工業,1108.TW
...,...,...,...,...,...
1794,9949,琉園,上櫃,文化創意業,9949.TWO
1795,9950,萬國通,上櫃,塑膠工業,9950.TWO
1796,9951,皇田,上櫃,電機機械,9951.TWO
1797,9960,邁達康,上櫃,運動休閒,9960.TWO


In [5]:
# LOAD DATA ##################
directory, filename = os.path.split(data_filepath)

# Check if the directory exists
if not os.path.exists(directory):
    # If not, create the directory
    os.makedirs(directory)
    print(f"Directory '{directory}' created.")

# load data
if os.path.exists(data_filepath):
    data_df = pd.read_feather(data_filepath)
    
else:  # Data file does not exist, download it from Yahoo Finance
    print('No current data file found, start to download')
    
    # Get stock symbols
    symbols = stock_symbol_df['yf_symbol'].to_list()

    # Download data
    data_df = yf.download(symbols, session=session, period="10y", progress=True)

    # Save data
    data_df.to_feather(data_filepath)

In [6]:
data_df.tail()  # Inspect data df

Adj Close                                                      \
              1101.TW    1102.TW    1103.TW    1104.TW 1108.TW    1109.TW   
Date                                                                        
2023-11-01  32.099998  40.000000  17.900000  25.850000   13.60  17.450001   
2023-11-02  32.349998  40.000000  17.950001  26.049999   13.95  17.650000   
2023-11-03  32.900002  40.250000  18.150000  26.150000   13.90  17.700001   
2023-11-06  32.799999  40.400002  18.400000  26.350000   13.90  17.750000   
2023-11-07  32.549999  40.400002  18.299999  26.250000   13.90  17.750000   

                                                        ...       Open  \
              1110.TW    1201.TW    1203.TW    1210.TW  ...   6856.TWO   
Date                                                    ...              
2023-11-01  18.150000  18.450001  42.400002  51.900002  ...  75.800003   
2023-11-02  18.350000  18.500000  42.250000  52.099998  ...  74.900002   
2023-11-03  18.350000  18.600000  42.099998  52.500000  ...  74.800003   
2023-11-06  18.400000  18.650000  43.250000  52.799999  ...  72.000000   
2023-11-07  18.450001  18.799999  42.599998  52.799999  ...  72.500000   

                         Volume           Adj Close      Close       High  \
              8476.TW  6856.TWO  8476.TW    6658.TW    6658.TW    6658.TW   
Date                                                                        
2023-11-01  63.500000  137336.0  42085.0        NaN        NaN        NaN   
2023-11-02  62.400002   92515.0  62695.0  40.000000  40.000000  41.000000   
2023-11-03  62.400002   87691.0  23654.0        NaN        NaN        NaN   
2023-11-06  61.599998   65948.0  41410.0  39.000000  39.000000  39.500000   
2023-11-07  61.299999   52159.0  59738.0  39.049999  39.049999  39.299999   

                  Low       Open     Volume  
              6658.TW    6658.TW    6658.TW  
Date                                         
2023-11-01        NaN        NaN        NaN  
2023-11-02  38.150002  39.200001  1102324.0  
2023-11-03        NaN        NaN        NaN  
2023-11-06  38.599998  39.150002   269875.0  
2023-11-07  38.650002  39.000000   143680.0  

[5 rows x 10806 columns]

### 更新資料

In [7]:
# Get date information
now = datetime.datetime.now(tz=pytz.timezone('Asia/Taipei'))
date_today = now.date()
last_day_data = data_df.index[-1].date()

In [8]:
date_today + datetime.timedelta(days=1)

datetime.date(2023, 11, 9)

In [9]:
if date_today > last_day_data:
    # The target day for data update
    target_day = date_today if now.time() > datetime.time(15, 00) else date_today + datetime.timedelta(days=-1)
    start_day = last_day_data #+ datetime.timedelta(days=1)
    end_day = target_day + datetime.timedelta(days=1)

    if start_day <= target_day:
       
        update_data_df = yf.download(['2330.TW'], start=start_day.strftime('%Y-%m-%d'), end=end_day.strftime('%Y-%m-%d'), progress=False)  # For test run

        # Update the data df and save to file
        if len(update_data_df.index) > 0:  # Double-check new data is available
            symbols = stock_symbol_df['yf_symbol'].to_list()
            update_data_df = yf.download(symbols, start=start_day.strftime('%Y-%m-%d'), end=end_day.strftime('%Y-%m-%d'), progress=True)
            data_df = pd.concat([data_df, update_data_df])
            data_df.to_feather(data_filepath)
        else:
            print('New data is not available. Maybe try later.')
    else:
        print('No update')

print(f'Last three days of the data: {data_df.tail(3).index.to_list()}')

[*********************100%%**********************]  1799 of 1799 completed


4 Failed downloads:
['8423.TWO', '2740.TWO', '3089.TWO', '8080.TWO']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-07 -> 2023-11-09)')



Last three days of the data: [Timestamp('2023-11-06 00:00:00'), Timestamp('2023-11-07 00:00:00'), Timestamp('2023-11-08 00:00:00')]


In [10]:
# Remove duplicate dates just in case
data_df = data_df.reset_index().drop_duplicates(subset=[('Date', '')], keep='first').set_index('Date')
data_df.to_feather(data_filepath)